In [1]:
import pandas as pd
import numpy as np
import datetime
from  datetime import datetime as dt

import os
import glob
from tqdm import tqdm
import math

import warnings
warnings.filterwarnings('ignore')

In [2]:
file_path = 'C:/Users/rian/Documents/2023/02.nonmun/01.data'

In [ ]:
# labels 데이터 병합
for p in tqdm([file_path + '/' + f + '/' for f in os.listdir(file_path) if f.split('.')[-1] != 'csv']):
    under1 = [p + l for l in os.listdir(p)]
    temp3_list = []
    temp1 = pd.DataFrame()
    # 연도별 유저 리스트
    for u1 in under1:
        under2 = [u1 + '/' + u for u in os.listdir(u1)]
        temp2 = pd.DataFrame()
        # 유저별 일자 리스트
        for u2 in under2:
            temp5 = pd.read_csv(glob.glob(u2 + '/*.csv')[0])
            temp5.rename(columns={'ts':'timestamp'}, inplace=True)
            temp5.set_index('timestamp', inplace=True)
            temp2 = pd.concat([temp2, temp5], axis=0)

        userId = pd.DataFrame([''.join([i for i in u1.split('/')[-1] if i.isdigit()])] * temp2.shape[0], index=temp2.index, columns=['userId'])
        temp2 = pd.concat((temp2, userId), axis=1)
        temp2.reset_index(drop=False, inplace=True)

        temp1 = pd.concat([temp1, temp2], axis=0, ignore_index=True)
    temp1 = temp1[['timestamp', 'userId'] + [c for c in temp1.columns if c not in ['timestamp', 'userId']]]
    temp1.to_csv(f'C:/Users/rian/Documents/2023/02.nonmun/02.result/{p.split("/")[-2]}_labels.csv', header=True, index=False)

In [3]:
# survey, info 병합
def merge_df(survey_df, info_df):
    try:
        survey_df['inputDt'] = pd.to_datetime(survey_df['inputDt'])
        survey_df['date'] = survey_df['inputDt'].apply(lambda x:x - datetime.timedelta(days=1) if x.hour < 5 else x)
        survey_df['date'] = pd.to_datetime(survey_df['date']).dt.date
    except:
        survey_df['startInput'] = pd.to_datetime(survey_df['startInput'])
        survey_df['endInput'] = pd.to_datetime(survey_df['endInput']) 
        
    survey_df.sort_values(by=['userId', 'date', 'amPm'], ignore_index=True, inplace=True)    
    survey_df['userId'] = survey_df['userId'].astype('str')
    survey_df['userId'] = survey_df['userId'].str.extract(r'(\d+)').astype('int')
    
    info_df['userId'] = info_df['userId'].astype('str')
    info_df['userId'] = info_df['userId'].str.extract(r'(\d+)').astype('int')
    info_df['startDt'] = pd.to_datetime(info_df['startDt'])
    info_df['endDt'] = pd.to_datetime(info_df['endDt'])
    
    result_df = info_df.merge(right=survey_df, on=['userId'], how='right')
    result_df.dropna(subset=['gender', 'age'], how='all', axis=0, inplace=True)
    return result_df

In [4]:
# 유저 데이터 read
# servey
user_survey_20189 = pd.read_csv('C:/Users/rian/Documents/2023/02.nonmun/01.data\\user_survey_2019_2018.csv')
# 2018, 2019 분리
user_survey_2018 = user_survey_20189[user_survey_20189['userId'] < 100].reset_index(drop=True)
user_survey_2019 = user_survey_20189[user_survey_20189['userId'] >= 100].reset_index(drop=True)
del user_survey_20189
user_survey_2020 = pd.read_csv('C:/Users/rian/Documents/2023/02.nonmun/01.data\\user_survey_2020.csv')

# info
user_info_20189 = pd.read_csv('C:/Users/rian/Documents/2023/02.nonmun/01.data\\user_info_2019_2018_updated.csv')
user_info_20189['userId'] = user_info_20189['userId'].str.extract(r'(\d+)').astype('int')
# 2018, 2019 분리
user_info_2018 = user_info_20189[(user_info_20189['userId'] < 100) & (user_info_20189['age'] < 40)].reset_index(drop=True)
user_info_2019 = user_info_20189[(user_info_20189['userId'] >= 100) & (user_info_20189['age'] < 40)].reset_index(drop=True)
del user_info_20189
user_info_2020 = pd.read_csv('C:/Users/rian/Documents/2023/02.nonmun/01.data\\user_info_2020.csv')
user_info_2020['userId'] = user_info_2020['userId'].str.extract(r'(\d+)').astype('int')

# id, 날짜에 맞추어 데이터 병합
lifelog_2018 = merge_df(user_survey_2018.copy(), user_info_2018.copy())
lifelog_2019 = merge_df(user_survey_2019.copy(), user_info_2019.copy())
lifelog_2020 = merge_df(user_survey_2020.copy(), user_info_2020.copy())

del user_survey_2018
del user_info_2018
del user_survey_2019
del user_info_2019
del user_survey_2020
del user_info_2020

# lifelog_2018.to_csv('C:/Users/rian/Documents/2023/02.nonmun/02.result/2018_tmp2.csv', header=True, index=False)
# lifelog_2019.to_csv('C:/Users/rian/Documents/2023/02.nonmun/02.result/2019_tmp2.csv', header=True, index=False)
# lifelog_2020.to_csv('C:/Users/rian/Documents/2023/02.nonmun/02.result/2020_tmp2.csv', header=True, index=False)

In [5]:
labels_path = 'C:/Users/rian/Documents/2023/02.nonmun/02.result'

In [6]:
# label 데이터 read
label_2018 = pd.read_csv(labels_path + '/' + 'dataset_2018_labels.csv')
label_2019 = pd.read_csv(labels_path + '/' + 'dataset_2019_labels.csv')
label_2020 = pd.concat([pd.read_csv(labels_path + '/' + 'user01-06_labels.csv'),
                        pd.read_csv(labels_path + '/' + 'user07-10_labels.csv'),
                        pd.read_csv(labels_path + '/' + 'user11-12_labels.csv'),
                        pd.read_csv(labels_path + '/' + 'user21-25_labels.csv'),
                        pd.read_csv(labels_path + '/' + 'user26-30_labels.csv')], axis=0)

In [7]:
label_2018['timestamp'] = pd.to_datetime(label_2018['timestamp'], unit='s')
label_2019['timestamp'] = pd.to_datetime(label_2019['timestamp'], unit='s')
label_2020['timestamp'] = pd.to_datetime(label_2020['timestamp'], unit='s')

In [8]:
# label 데이터 집계
def count_df(df):
    dict_temp = {}
    i = 0
    
    for Id, content in df.groupby('userId'):
        content.set_index('timestamp', inplace=True)
        
        for d, content2 in content.groupby(by=content.index.date):
            dict_temp[i] = [d, Id]
            
            for c in ['sleep', 'personal_care', 'work', 'study', 'household', 'care_housemem', 'recreation_media', 'entertainment',
                      'outdoor_act', 'hobby', 'recreation_etc', 'shop', 'communitiy_interaction', 'travel', 'meal', 'socialising']:
                dict_temp[i] += [content2[content2['action'] == c].shape[0]]
                
            try:
                # ALONE
                dict_temp[i] += [content2[content2['condition'] <= 0].shape[0]]
                # with
                dict_temp[i] += [content2[content2['condition'] > 0].shape[0]]
            except:
                # ALONE
                dict_temp[i] += [content2[content2['condition'] == 'ALONE'].shape[0]]
                # with
                dict_temp[i] += [content2[content2['condition'] != 'ALONE'].shape[0]]
                
            # with_families
            dict_temp[i] += [content2[content2['conditionSub1Option'] == 1.0].shape[0]]
            # with_friends
            dict_temp[i] += [content2[content2['conditionSub1Option'] == 2.0].shape[0]]
            # with_colleagues
            dict_temp[i] += [content2[content2['conditionSub1Option'] == 3.0].shape[0]]
            # acquaintances
            dict_temp[i] += [content2[content2['conditionSub1Option'] == 4.0].shape[0]]
            # with_unknown
            dict_temp[i] += [content2[content2['conditionSub1Option'].isnull()].shape[0]]
            
            # place
            for p in ['home', 'workplace', 'restaurant', 'outdoor', 'other_indoor']:
                dict_temp[i] += [content2[content2['place'] == p].shape[0]]
            for eP in range(1, 8):
                dict_temp[i] += [content2[content2['emotionPositive'] == float(eP)].shape[0]]
            for eT in range(1, 8):
                dict_temp[i] += [content2[content2['emotionTension'] == float(eT)].shape[0]]
                
            # in_vehicle
            dict_temp[i] += [content2[content2['activity'] == 0].shape[0]]
            # on_bicycle
            dict_temp[i] += [content2[content2['activity'] == 1].shape[0]]
            # on_foot
            dict_temp[i] += [content2[content2['activity'] == 2].shape[0]]
            # still
            dict_temp[i] += [content2[content2['activity'] == 3].shape[0]]
#             # unknown
#             dict_temp[i] += [content2[content2['activity'] == 4].shape[0]]
#             # tilting
#             dict_temp[i] += [content2[content2['activity'] == 5].shape[0]]
#             # walking
#             dict_temp[i] += [content2[content2['activity'] == 7].shape[0]]
#             # running
#             dict_temp[i] += [content2[content2['activity'] == 8].shape[0]]
            i += 1

    count_labels = pd.DataFrame(dict_temp).T
    count_labels.columns = ['date', 'userId', 'sleep', 'personal_care', 'work', 'study', 'household',
                            'care_housemem', 'recreation_media', 'entertainment', 'outdoor_act',
                            'hobby', 'recreation_etc', 'shop', 'communitiy_interaction', 'travel',
                            'meal', 'socialising', 'ALONE', 'WITH', 'with_families', 'with_friends',
                            'with_colleagues', 'acquaintances', 'with_unknown', 'home', 'workplace',
                            'restaurant', 'outdoor', 'other_indoor',
                            'emotionPositive1', 'emotionPositive2', 'emotionPositive3', 'emotionPositive4',
                            'emotionPositive5', 'emotionPositive6', 'emotionPositive7',
                            'emotionTension1', 'emotionTension2', 'emotionTension3', 'emotionTension4',
                            'emotionTension5', 'emotionTension6', 'emotionTension7',
                            'in_vehicle', 'on_bicycle', 'on_foot', 'still'] # , 'unknown', 'tilting', 'walking', 'running']
    
    count_labels['household'] = count_labels['household'] + count_labels['care_housemem']
    count_labels.drop(columns=['care_housemem'], inplace=True)
    count_labels['userId'] = count_labels['userId'].astype('str')
    count_labels['date'] = pd.to_datetime(count_labels['date'])
    
    return count_labels

In [9]:
# 연도별, 유저별 label 집계
count_2018 = count_df(label_2018.copy())
count_2019 = count_df(label_2019.copy())
count_2020 = count_df(label_2020.copy())

In [10]:
index_ = ['userId', 'gender', 'age', 'date']
amCol = ['sleep_quality', 'sleepProblem', 'dream', 'amCondition', 'amEmotion']
pmCol = ['pmEmotion', 'pmStress']
comCol = ['caffeine', 'alcohol']

In [11]:
# survey 집계
def survey_count(df):
    try:
        df['amCaffeine'].fillna(df['pmCaffeine'], inplace=True)
        df.rename(columns={'amCaffeine':'caffeine'}, inplace=True)
    except:
        pass
    
    df.rename(columns={'sleep':'sleep_quality'}, inplace=True)
    
    df['date'] = pd.to_datetime(df['date'])    
    df['userId'] = df['userId'].astype('str')
    
    for c in comCol:
        df.loc[df[df[c].isnull() == False].index, [c]] = 1
        df[c].fillna(0, inplace=True)

    result = pd.DataFrame(columns=['date', 'userId', 'gender', 'age',
                                   'sleep_quality', 'sleepProblem', 'dream',
                                   'amCondition', 'amEmotion', 'pmEmotion', 'caffeine', 'alcohol',
                                   'pmStress'])

    for g, content in df.groupby(by=index_):
        temp = {}
        for i in index_:
            temp[i] = content.iloc[0][i]

        amContent = content[content['amPm'] == 'am']
        for a in amCol:
            try:
                temp[a] = amContent.iloc[0][a]
            except:
                temp[a] = np.nan
                
        pmContent = content[content['amPm'] == 'pm']
        for p in pmCol:
            try:
                temp[p] = pmContent.iloc[0][p]
            except:
                temp[p] = np.nan
                
        del amContent
        del pmContent
        
        for c in comCol:
            temp[c] = 0 if content[c].sum() <= 0 else 1

        result = result.append(temp, ignore_index=True)
        
    return result

In [ ]:
# survey 집계
lifelog_data_2018 = survey_count(lifelog_2018.copy()).merge(count_2018.copy(), on=['date', 'userId'], how='inner').dropna()
lifelog_data_2019 = survey_count(lifelog_2019.copy()).merge(count_2019.copy(), on=['date', 'userId'], how='inner').dropna()
lifelog_data_2020 = survey_count(lifelog_2020.copy()).merge(count_2020.copy(), on=['date', 'userId'], how='inner').dropna()

In [ ]:
lifelog_data_2018.to_csv(r'C:\Users\rian\Documents\2023\02.nonmun\02.result\lifelog_data_2018.csv', index=False)
lifelog_data_2019.to_csv(r'C:\Users\rian\Documents\2023\02.nonmun\02.result\lifelog_data_2019.csv', index=False)
lifelog_data_2020.to_csv(r'C:\Users\rian\Documents\2023\02.nonmun\02.result\lifelog_data_2020.csv', index=False)